# FGSM

In [ ]:
import torch
from torch.autograd import Variable

def fgsm_attack(model, x, y, epsilon):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    x = Variable(x.to(device), requires_grad=True)
    y = Variable(y.to(device))
    
    # Set the model to evaluation mode
    model.eval()

    # Set requires_grad attribute of tensor x
    x.requires_grad = True

    # Forward pass through the model
    output = model(x)

    # Calculate the loss
    loss = torch.nn.functional.cross_entropy(output, y)

    # Calculate the gradients of the loss w.r.t. the input image
    model.zero_grad()
    loss.backward()
    gradient = x.grad.data

    # Calculate the sign of the gradient
    sign_gradient = gradient.sign()

    # Generate the perturbation by scaling the sign of the gradient with epsilon
    perturbation = epsilon * sign_gradient

    # Create the adversarial example by adding the perturbation to the input image
    adversarial_example = x + perturbation

    # Clip the pixel values of the adversarial example to ensure that they remain within the valid range
    adversarial_example = torch.clamp(adversarial_example, 0, 1)

    # Return the adversarial example
    return adversarial_example

In [ ]:
import torchvision.transforms as transforms
import os
from PIL import Image

def do_fgsm_attack(model, indir, outdir, epsilon):
    
    if not os.path.exists(outdir):
        os.makedirs(outdir)
            
    if torch.cuda.is_available():
        model.cuda()
    model.eval()

    file_list = os.listdir(indir)
    images = [file for file in file_list if file.endswith(".jpg")]

    for filepath in images:
        
        # Load the input image
        image = Image.open(indir+filepath)

        # Preprocess the image
        transform = transforms.Compose([
            transforms.Resize(640),
            transforms.ToTensor(),
        ])
        image = transform(image).unsqueeze(0)

        # Define the true label for the input image
        label = torch.tensor([0])

        # Perform the FGSM attack
        perturbed_image = fgsm_attack(model, image, label, epsilon)

        # Save
        perturbed_image = perturbed_image.squeeze(0).detach().cpu()
        perturbed_image = transforms.ToPILImage()(perturbed_image)
        perturbed_image.save(outdir+filepath)


# I-FGSM

In [ ]:
import torch
from torch.autograd import Variable

def i_fgsm_attack(model, image, target, epsilon, alpha, num_iter):
        
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image = Variable(image.to(device), requires_grad=True)
    target = Variable(target.to(device), requires_grad=False)
    
    # Clone the image tensor to avoid changing the original image
    adversarial_image = image.clone().detach().requires_grad_(True)

    for i in range(num_iter):
        # Forward pass to get the model prediction
        output = model(adversarial_image)
        loss = torch.nn.functional.cross_entropy(output, target)

        # Calculate gradients
        model.zero_grad()
        loss.backward()
        gradient = adversarial_image.grad.detach()

        # Add perturbation
        perturbation = alpha * torch.sign(gradient)
        adversarial_image = adversarial_image + perturbation

        # Clip the perturbation to ensure it is within epsilon bounds
        clipped_perturbation = torch.clamp(adversarial_image - image, min=-epsilon, max=epsilon)
        adversarial_image = torch.clamp(image + clipped_perturbation, min=0, max=1).detach().requires_grad_(True)

    return adversarial_image


In [ ]:
import torchvision.transforms as transforms
import os
from PIL import Image

def do_i_fgsm_attack(model, indir, outdir, epsilon, alpha, num_iter):
    
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    if torch.cuda.is_available():
        model.cuda()
    model.eval()

    file_list = os.listdir(indir)
    images = [file for file in file_list if file.endswith(".jpg")]

    for filepath in images:
        # Load the input image
        image = Image.open(indir+filepath)

        # Preprocess the image
        transform = transforms.Compose([
            transforms.Resize(640),
            transforms.ToTensor(),
        ])
        image = transform(image).unsqueeze(0)

        # Perform the I-FGSM attack
        perturbed_image = i_fgsm_attack(model, image, torch.tensor([0]), epsilon, alpha, num_iter)

        # Save
        perturbed_image = perturbed_image.squeeze(0).detach().cpu()
        perturbed_image = transforms.ToPILImage()(perturbed_image)
        perturbed_image.save(outdir+filepath)


# MI-FGSM

In [ ]:
import torch
from torch.autograd import Variable

def mi_fgsm_attack(model, image, target, epsilon, mu, num_iter):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image = Variable(image.to(device), requires_grad=True)
    adv_image = Variable(image.to(device), requires_grad=True)
    target = Variable(target.to(device))

    # Iterate for the given number of iterations
    for i in range(num_iter):
        # Compute the loss and gradient of the loss with respect to the input
        loss = torch.nn.functional.cross_entropy(model(adv_image), target)
        grad = torch.autograd.grad(loss, adv_image)[0]
        
        # Update the perturbation using the MI-FGSM formula
        adv_image = adv_image.detach() + mu * grad.sign() + (epsilon / num_iter) * grad.sign()
        adv_image = torch.max(torch.min(adv_image, image + epsilon), image - epsilon).clamp(0, 1)
        adv_image = adv_image.detach().requires_grad_(True)

    # Return the adversarial image
    return adv_image

In [ ]:
import torchvision.transforms as transforms
import os
from PIL import Image

def do_mi_fgsm_attack(model, indir, outdir, epsilon, mu, num_iter):
    
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    if torch.cuda.is_available():
        model.cuda()
    model.eval()

    file_list = os.listdir(indir)
    images = [file for file in file_list if file.endswith(".jpg")]

    for filepath in images:
        # Load the input image
        image = Image.open(indir+filepath)

        # Preprocess the image
        transform = transforms.Compose([
            transforms.Resize(500),
            transforms.ToTensor(),
        ])
        image = transform(image).unsqueeze(0)

        # Load the true label
        label = torch.tensor([0])

        # Perform the MI-FGSM attack
        perturbed_image = mi_fgsm_attack(model, image, label, epsilon, mu, num_iter)

        # Save
        perturbed_image = perturbed_image.squeeze(0).detach().cpu()
        perturbed_image = transforms.ToPILImage()(perturbed_image)
        perturbed_image.save(outdir+filepath)


# PGD

In [ ]:
import torch
from torch.autograd import Variable

def pgd_attack(model, image, target_class, epsilon, alpha, num_iter):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image = Variable(image.to(device), requires_grad=True)
    adv_image = Variable(image.to(device), requires_grad=True)
    target_class = Variable(target_class.to(device))
    
    # Initialize the perturbation δ to zero
    delta = torch.zeros_like(image, requires_grad=True)
    
    for i in range(num_iter):
        # Calculate the gradient of the loss function J with respect to the input image x
        output = model(image + delta)
        loss = torch.nn.functional.cross_entropy(output, target_class)
        loss.backward()
        gradient = delta.grad.detach()
        
        # Add a scaled version of the gradient to the perturbation
        delta.data = delta + alpha * torch.sign(gradient)
        
        # Clip the perturbation δ so that its L∞ norm is at most ϵ
        delta.data = torch.max(torch.min(delta, torch.tensor(epsilon)), torch.tensor(-epsilon))
        
        # Update the adversarial example by adding the perturbation to the input image
        adv_image = torch.clamp(image + delta, 0, 1).detach_()
        
        # Zero out the gradient
        delta.grad.zero_()
        
    return adv_image


In [ ]:
import torchvision.transforms as transforms
import os
from PIL import Image

def do_pgd_attack(model, indir, outdir, epsilon, alpha, num_iter):
    
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    if torch.cuda.is_available():
        model.cuda()
    model.eval()

    file_list = os.listdir(indir)
    images = [file for file in file_list if file.endswith(".jpg")]

    for filepath in images:
        # Load the input image
        image = Image.open(indir+filepath)

        # Preprocess the image
        transform = transforms.Compose([
            transforms.Resize(500),
            transforms.ToTensor(),
        ])
        image = transform(image).unsqueeze(0)

        # Perform the PGD attack
        perturbed_image = pgd_attack(model, image, torch.tensor([0]), epsilon, alpha, num_iter)

        # Save
        perturbed_image = perturbed_image.squeeze(0).detach().cpu()
        perturbed_image = transforms.ToPILImage()(perturbed_image)
        perturbed_image.save(outdir+filepath)


# Execute adversarial atack

In [ ]:
import torchvision.models as models
import math
import time

# Load the pre-trained model
model = models.efficientnet_v2_s(pretrained=True)
epsilons = [0.01, 0.05, 0.1, 0.2, 0.3]
indir = '/home/culee/pytest/ShipClassification/dataset/val/'
print('Model: EfficientNetv2s')

# FGSM
for epsilon in epsilons:
    outdir = '/home/culee/pytest/ShipClassification/dataset/FGSM_' + str(epsilon) +'/'
    print(outdir)
    start = time.time()
    do_fgsm_attack(model, indir, outdir, epsilon)
    end = time.time()
    print(f"{end - start:.5f} sec")
    
# I-FGSM
for epsilon in epsilons:
    outdir = '/home/culee/pytest/ShipClassification/dataset/I_FGSM_' + str(epsilon) +'/'
    print(outdir)
    start = time.time()
    do_i_fgsm_attack(model, indir, outdir, epsilon, 0.01, 20)
    end = time.time()
    print(f"{end - start:.5f} sec")

# MI-FGSM
for epsilon in epsilons:
    outdir = '/home/culee/pytest/ShipClassification/dataset/MI_FGSM_' + str(epsilon) +'/'
    print(outdir)
    start = time.time()
    do_mi_fgsm_attack(model, indir, outdir, epsilon, 0.001, 20)
    end = time.time()
    print(f"{end - start:.5f} sec")

# PGD
for epsilon in epsilons:
    outdir = '/home/culee/pytest/ShipClassification/dataset/PGD_' + str(epsilon) +'/'
    print(outdir)
    start = time.time()
    do_pgd_attack(model, indir, outdir, epsilon, 0.01, 20)
    end = time.time()
    print(f"{end - start:.5f} sec")

